# 3.5 Complex Recursion

Recursive rules can be incredible powerful, able to be taken even further than the examples shown in 3.3 & 3.4 by incorporating calculations.

Rules like these can be used to solve traditionally challenging data problems such as the bill of materials, where an aggregated cost must be calculated for hierarchical parts.

## Example

Critically for this example, we want to calculate the aggregated cost at each point in the tree, not just the highest level

This is the most conceptually complex exercise in this workshop, so it's important to work up one layer at a time.

It can help to visualise this problem, so run the cell below then click here to [explore the data](http://localhost:12110/console/datastores/explore?datastore=default&query=SELECT%20%3FS%20%3FP%20%3FO%0AWHERE%20%7B%0A%20%20%20%20%3FS%20%3FP%20%3FO%0A%7D) and change the layout to **Breadth First**.

In [1]:
import requests

cr_data = """
@prefix : <https://rdfox.com/example#> .

:compoundB :contains :compoundA,
                :part4 .

:part4 :hasCost 10 .

:compoundA :contains :part1,
                :part2 ,
                :part3 .

:part1 :hasCost 1 .
:part2 :hasCost 1 .
:part3 :hasCost 1 .

"""

# Set up the SPARQL endpoint
rdfox_server = "http://localhost:12110"

# Helper function to raise exception if the REST endpoint returns an unexpected status code
def assert_response_ok(response, message):
    if not response.ok:
        raise Exception(
            message + "\nStatus received={}\n{}".format(response.status_code, response.text))
    
# Clear data store
clear_response = requests.delete(
    rdfox_server + "/datastores/default/content?facts=true&axioms&rules")
assert_response_ok(clear_response, "Failed to clear data store.")

# Add data
payload = {'operation': 'add-content-update-prefixes'}
data_response = requests.patch(
    rdfox_server + "/datastores/default/content", params=payload, data=cr_data)
assert_response_ok(data_response, "Failed to add facts to data store.")


The challenge here is that we wish to calculate the aggregate cost at each compound point - both for the collections of parts, and for the collection of sub-compounds.

However, RDFox will not accept aggregates (or negation) that rely on recursive results (see 2.2), so we cannot write a simple recursive rule set.

To demonstrate that, below is a rule that might naively be written to solve this.

When trying to import this rule, RDFox will throw a stratification error.

In [2]:
unstratified_rules = """

[?compound, :hasCost, ?totalCost] :-
    AGGREGATE(
        [?compound, :contains, ?thing],
        [?thing, :hasCost, ?cost]
        ON ?compound
        BIND SUM(?cost) AS ?totalCost
    ).

"""

# Add rules
rules_response = requests.post(rdfox_server + "/datastores/default/content", data=unstratified_rules)
assert_response_ok(rules_response, "Failed to add rule.")

print(rules_response.text)

Exception: Failed to add rule.
Status received=400
RuleCompilationException: The program is not stratified because these components of the dependency graph contain cycles through negation and/or aggregation:
======== COMPONENT 1 ========
    <https://rdfox.com/example/hasCost>[?compound, ?totalCost] :- AGGREGATE(<https://rdfox.com/example/contains>[?compound, ?thing], <https://rdfox.com/example/hasCost>[?thing, ?cost] ON ?compound BIND SUM(?cost) AS ?totalCost) .
========================================================================================================================


## Solving the Bill of Materials

To get around this, we must perform the aggregates in a deterministic, stratified way - we must state the order in which it will occur.

One way to do this is to create an order for the parts and components, and use `BIND` to sum the running total from one part to the next. This way no recursive rules incorporate an aggregate.


## Ordering entities for an arbitrary compound node

To order arbitrary nodes, we first have to find each node's relative position to one another (whether it is before or after), then give them a specific position within the list by putting node next to one another where there is no other node in between.

In [4]:
ordering_rules = """

[?thing1, :hasGreaterThing, ?thing2] :-
    [?compound, :contains, ?thing1],
    [?compound, :contains, ?thing2],
    FILTER (?thing2 > ?thing1).

[?thing1, :hasNextThing, ?thing2] :-
    [?thing1, :hasGreaterThing, ?thing2],
    NOT EXISTS ?thing3 IN ( 
        [?thing1, :hasGreaterThing, ?thing3], 
        [?thing3, :hasGreaterThing, ?thing2]
    ).

"""
# Clear data store
clear_response = requests.delete(
    rdfox_server + "/datastores/default/content?facts=true&axioms&rules")
assert_response_ok(clear_response, "Failed to clear data store.")

# Add rules
rules_response = requests.post(rdfox_server + "/datastores/default/content", data=ordering_rules)
assert_response_ok(rules_response, "Failed to add rule.")

# Add data
payload = {'operation': 'add-content-update-prefixes'}
data_response = requests.patch(
    rdfox_server + "/datastores/default/content", params=payload, data=cr_data)
assert_response_ok(data_response, "Failed to add facts to data store.")

### Progress so far...

[View the results of these rules here](http://localhost:12110/console/datastores/explore?datastore=default&query=SELECT%20%3FS%20%3FP%20%3FO%0AWHERE%20%7B%0A%20%20%20%20%3FS%20%3FP%20%3FO%0A%7D) and **change the layout to Breadth first**.

Use the bulb to highlight the facts that have been inferred.

## Finding the first and last members of the chain

Next we need to identify the first and last members of the chains we've created.

It may not be immediately clear why we do this, but they will both be helpful in the final step.

In [4]:
end_point_rules = """

[?compound, :hasFirstThing, ?firstThing] :-
    [?compound, :contains, ?firstThing],
    NOT EXISTS ?something IN ([?something, :hasGreaterThing, ?firstThing]).

[?compound, :hasLastThing, ?lastThing] :-
    [?compound, :contains, ?lastThing],
    NOT EXISTS ?something IN ([?lastThing, :hasGreaterThing, ?something]).

"""

# Add rules
rules_response = requests.post(rdfox_server + "/datastores/default/content", data=end_point_rules)
assert_response_ok(rules_response, "Failed to add rule.")

### Progress so far...

[View the results of these rules here](http://localhost:12110/console/datastores/explore?datastore=default&query=SELECT%20%3FS%20%3FP%20%3FO%0AWHERE%20%7B%0A%20%20%20%20%3FS%20%3FP%20%3FO%0A%7D) and **change the layout to Breadth first**.

Use the bulb to highlight the facts that have been inferred.

## Calculating a running total

Now that all the pieces are in place, we can recursively calculate a running total.

The running count begins at the first thing in the compound and is passed up to the compound node once it has reached the last thing.

In [5]:
running_total_rules = """

[?firstThing, :hasRunningTotal, ?cost] :-
    [?compound, :hasFirstThing, ?firstThing],
    [?firstThing, :hasCost, ?cost].

[?thing2, :hasRunningTotal, ?runningTotal] :-
    [?thing1, :hasNextThing, ?thing2],
    [?thing1, :hasRunningTotal, ?oldRunningTotal],
    [?thing2, :hasCost, ?cost],
    BIND (?oldRunningTotal + ?cost AS ?runningTotal).

[?compound, :hasCost, ?compoundTotal] :-
    [?compound, :hasLastThing, ?lastThing],
    [?lastThing, :hasRunningTotal, ?compoundTotal].

"""

# Add rules
rules_response = requests.post(rdfox_server + "/datastores/default/content", data=running_total_rules)
assert_response_ok(rules_response, "Failed to add rule.")

### Check the results!

[View the results of these rules here](http://localhost:12110/console/datastores/explore?datastore=default&query=SELECT%20%3FS%20%3FP%20%3FO%0AWHERE%20%7B%0A%20%20%20%20%3FS%20%3FP%20%3FO%0A%7D) and **change the layout to Breadth first**.

Use the bulb to highlight the facts that have been inferred.

## Explainability

As complex as the emerging results may be, rules simply offer a series of logical inferences, and this means that every result is examplable.

We saw simple example of this in 3.1, but this proof tree is significantly more intricate.

[Click here](http://localhost:12110/console/datastores/explain?datastore=default&fact=%5B%3Chttps%3A%2F%2Frdfox.com%2Fexample%23compoundB%3E%2C%20%3Chttps%3A%2F%2Frdfox.com%2Fexample%23hasCost%3E%2C%2013%5D) to explain the cost of `compoundB`. Results can be explained in the **Explain** tab of the RDFox Console.

## Exercise

Complete the rule `3_5-ComplexRecursionRules.dlog` in the `rules` folder so that the query below can be used to directly find the highest-level compound nodes that each represent a separate bill of materials.

**To do this**, must also write a rule to account for compound nodes that only have 1 child.

In [15]:
cr_sparql = """
SELECT ?thing ?compoundTotal
WHERE {
    ?thing a :TopLevelCompound ;
        :hasCost ?compoundTotal .
}
"""

Here is a representative sample of the data in `3_5-ComplexRecursionData.ttl`.

In [16]:
sample_data = """
@prefix : <https://rdfox.com/example#> .

:compound001 a :Compound ;
        :contains :compound002,
                :part4 .

:compound002 a :Compound ;
        :contains :compound003,
                :part1,
                :part2 ,
                :part3 .

:compound010 a :Compound ;
        :contains :part16 .                

:part1 :hasCost 3 .
:part2 :hasCost 2 .
:part3 :hasCost 9 .
:part4 :hasCost 7 .
:part16 :hasCost 1 .

"""

### Check your work

Run the query below to verify the results.

In [20]:
# Clear data store
clear_response = requests.delete(
    rdfox_server + "/datastores/default/content?facts=true&axioms&rules")
assert_response_ok(clear_response, "Failed to clear data store.")

# Get and add data
with open("../data/3_5-ComplexRecursionData.ttl", "r") as file:
    data = file.read()
payload = {'operation': 'add-content-update-prefixes'}
data_response = requests.patch(
    rdfox_server + "/datastores/default/content", params=payload, data=data)
assert_response_ok(data_response, "Failed to add facts to data store.")

# Get and add rules
with open("../rules/3_5-ComplexRecursionRules.dlog", "r") as rule_file:
    datalog_rule = rule_file.read()
response = requests.post(rdfox_server + "/datastores/default/content", data=datalog_rule)
assert_response_ok(response, "Failed to add rule.")

# Issue select query
response = requests.get(
    rdfox_server + "/datastores/default/sparql", params={"query": cr_sparql})
assert_response_ok(response, "Failed to run select query.")
print('\n=== Top Level Compounds & Totals ===')
print(response.text)


=== Switches Requiring Maintenance ===
?thing	?compoundTotal
<https://rdfox.com/example#compound001>	70
<https://rdfox.com/example#compound009>	74
<https://rdfox.com/example#compound017>	35
<https://rdfox.com/example#compound020>	64



### Visualise the results

The power in these results is much clearer when visualised.

Open them in the RDFox Explorer [here](http://localhost:12110/console/datastores/explore?datastore=default&query=SELECT%20%3FendPerson%20%3Fperson%0AWHERE%20%7B%0A%0A%20%20%20%20%3FpathNode%20%3ApathTo%20%3FendPerson%3B%0A%20%20%20%20%20%20%20%20%20%20%20%20%3AhasPathLength%20%3FpathLength%3B%0A%20%20%20%20%20%20%20%20%20%20%20%20%3ApathIncludes%20%3Fperson%20.%0A%0A%20%20%20%20%3Fperson%20%3AhopsAwayFromAlice%20%3FhopsAwayFromAlice.%0A%0A%7D%20ORDER%20BY%20DESC%28%3FpathLength%29%20ASC%28%3FendPerson%29%20ASC%28%3FhopsAwayFromAlice%29) and **change the layout to Breadth First**.

If you can separate 4 distinct trees, that's a good sign!

## You should see...

=== Top Level Compounds & Totals ===
|?thing|?compoundTotal|
|-----------|-------------|
|<https://rdfox.com/example#compound001>|	70|
|<https://rdfox.com/example#compound009>|	74|
|<https://rdfox.com/example#compound017>|	35|
|<https://rdfox.com/example#compound020>|	64|